## Cloning the dataset

In [1]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 4590, done.
remote: Total 4590 (delta 0), reused 0 (delta 0), pack-reused 4590 (from 1)
Receiving objects: 100% (4590/4590), 186.72 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (4155/4155), done.


In [2]:
%cd Face-Mask-Detection/dataset

/content/Face-Mask-Detection/dataset


In [3]:
!ls with_mask -l | wc -l

2166


In [4]:
!ls without_mask -l | wc -l

1931


In [5]:
%cd /content/

/content


In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import StandardScaler

dataset_path = "Face-Mask-Detection/dataset/"

def extract_features(image):
    image = cv2.resize(image, (128, 128))

    # HOG
    hog_feat = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    # LBP
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)

    # Combine HOG + LBP hist
    features = np.hstack((hog_feat, lbp_hist))
    return features

# Load dataset
X = []
y = []

for label, category in enumerate(['with_mask/', 'without_mask/']):
    for filename in os.listdir(dataset_path + category):
        img_path = dataset_path + category + filename
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        features = extract_features(img)
        X.append(features)
        y.append(label)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Normalize features (recommended)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
# SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train SVM
svm = SVC(kernel='poly', C=1.0, random_state=42)
svm.fit(X_train, y_train)

# Predict on test set
y_pred = svm.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 91.33%


In [9]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train XGBoost Classifier
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)

# Predict on test set
y_pred = xgb.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 92.67%
